# Predicting formation factor

The example explains formation factor calculations on a cubic network. Note that formation factor calcualtion for an extracted network from PoreSpy follows similar steps in assigning phase, algorithm and calculating the effective property.

In [1]:
import numpy as np
import openpnm as op
%config InlineBackend.figure_formats = ['svg']
np.random.seed(10)
%matplotlib inline
np.set_printoptions(precision=5)

### Create a random cubic network

In [2]:
np.random.seed(10)
pn = op.network.Cubic(shape=[15, 15, 15], spacing=1e-6)
pn.add_model_collection(op.models.collections.geometry.spheres_and_cylinders)
pn.regenerate_models()

### Create phase object

It is assumed that a generic phase is the phase difffusing through the porous medium. As formation factor is the porous medium property and not the phase property, any other phase asumming a diffusion coefficient value can be used as the phase.

In [3]:
phase = op.phase.Phase(network=pn)
phase['pore.diffusivity']=1
phase.add_model_collection(op.models.collections.physics.basic)
phase.regenerate_models()

### Apply fickian diffusion

To calculate formation factor in x direction, a constant concentration boundary condition is applied on the left and right side of the network. Note that a similar procedure can be followed to find the formation factor in y and z directions.

In [4]:
inlet = pn.pores('left')
outlet = pn.pores('right')
Diff = op.algorithms.FickianDiffusion(network=pn, phase=phase)
Diff.set_value_BC(pores=inlet, values=1)
Diff.set_value_BC(pores=outlet, values=0)
Diff.run()

{'pore.concentration': SteadyStateSolution([1., 1., 1., ..., 0., 0., 0.])}

### Calculate formation factor

Calculate the formation factor, which is the inverse of effective diffusivity of the medium. Assuming concentration difference and diffusivity of 1, we have a simplified equation:

In [5]:
# NBVAL_IGNORE_OUTPUT
R = Diff.rate(pores=inlet, mode='group')[0]
A = op.topotools.get_domain_area(pn, inlets=inlet, outlets=outlet)
L = op.topotools.get_domain_length(pn, inlets=inlet, outlets=outlet)
D_eff = R * L / (A)
F = 1/D_eff
print(f'Formation factor is: {F:.2f}')

------------------------------------------------------------
    SOURCE     : openpnm.topotools._topotools.get_domain_area 
    TIME STAMP : 2022-07-13 11:35:52,104    
------------------------------------------------------------
------------------------------------------------------------
             boundary pores were not added 
    SOURCE     : openpnm.topotools._topotools.get_domain_length 
    TIME STAMP : 2022-07-13 11:35:52,119    
------------------------------------------------------------


Formation factor is: 23.20
